In [15]:
import pandas as pd
import csv
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
# Can run with either the tree data or the tree data plus PLUTO data
# data = pd.read_csv('data/parsed_data.csv')
data = pd.read_csv('data/parsed_pluto_tree_data.csv')

# Prepare Data

In [3]:
# Encode tree species as their frequency count rather than one hot encoding, since there are 100s of species
# Loss of info, but it's a tradeoff
data.spc_latin = data.spc_latin.map(data.spc_latin.value_counts()) 

In [4]:
# Encode borough as number
borough_dict = {"Manhattan":1, "Brooklyn": 2, "Queens": 3, "Bronx":4, "Staten Island": 5}
# data.borough = data.borough.map(borough_dict) 
data["borough"] = data["borough"].map(borough_dict)

In [5]:
data

,Unnamed: 0,latitude,longitude,borough,zipcode,spc_latin,tree_diameter,wires,sidew_crack_raise,latBin,lonBin,lonDistance,latDistance,avg_health_round,avg_health,landuseavg,numfloorsavg
0,0,40.748590,-73.984927,1.0,10001,470,4,0,0,40.7485,-73.9855,0.000573,0.000090,3.0,3.0,5.0,34.0
1,1,40.748835,-73.985520,1.0,10001,46,3,0,0,40.7485,-73.9855,0.000020,0.000335,3.0,3.0,5.0,34.0
2,2,40.748854,-73.985564,1.0,10001,683,5,0,0,40.7485,-73.9855,0.000064,0.000354,3.0,3.0,5.0,34.0
3,3,40.748801,-73.985436,1.0,10001,470,4,0,0,40.7485,-73.9855,0.000064,0.000301,3.0,3.0,5.0,34.0
4,4,40.748835,-73.985520,1.0,10001,470,3,0,0,40.7485,-73.9855,0.000020,0.000335,3.0,3.0,5.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3087,3087,40.728631,-73.851211,3.0,11375,32,17,0,0,40.7285,-73.8515,0.000289,0.000131,2.0,2.0,6.0,20.0
3088,3088,40.728506,-73.851632,3.0,11375,57,4,0,0,40.7285,-73.8515,0.000132,0.000006,2.0,2.0,6.0,20.0
3089,3089,40.728506,-73.851632,3.0,11375,57,2,0,0,40.7285,-73.8515,0.000132,0.000006,2.0,2.0,6.0,20.0
3090,3090,40.728506,-73.851632,3.0,11375,57,2,0,0,40.7285,-73.8515,0.000132,0.000006,2.0,2.0,6.0,20.0


In [19]:
# Num unique features
print(data['tree_diameter'].unique())
print(len(data['tree_diameter'].unique()))

print(data['spc_latin'].unique())
print(len(data['spc_latin'].unique()))

print(data['wires'].unique())
print(len(data['wires'].unique()))

print(data['sidew_crack_raise'].unique())
print(len(data['sidew_crack_raise'].unique()))

print(sorted(data['numfloorsavg'].unique()))
print(len(data['numfloorsavg'].unique()))

print(data['landuseavg'].unique())
print(len(data['landuseavg'].unique()))

print(len(data['latBin'].unique()))
print(len(data['lonBin'].unique()))

[ 4  3  5 11 10 15 17 16  6 14 13  0  7  2 12 18 19  8 20  1  9 24 26 22
 23 28 25 27 33 39 43 41 21 29 40 36 32 34 31 35]
40
[470  46 683  57 420   2  32  40 204  16  26  59  25  69  24 113   3 299
   9  15   7  36   5  11   6  58  37  12  63  18  17  10   1  13   4   8]
36
[0 1]
2
[0 1]
2
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 44.0, 45.0, 46.0, 47.0, 49.0, 50.0, 51.0, 57.0, 60.0]
52
[ 5.  3.  8.  6.  2.  4.  7. 10.  9.  1. 11.]
11
109


In [12]:
# Was using this when thinking about using more continuous data, but ended up using basically all categorical data

# Scale/Standardize data
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaler.fit(data) 
# data_scaled = pd.DataFrame(scaler.transform(data),columns =  )
# data_scaled

In [146]:
# Choose which features to use when running the models
X_train, X_test, y_train, y_test = train_test_split(data[['zipcode', 'spc_latin', 'tree_diameter', 'latBin', 'lonBin', 'wires', 'sidew_crack_raise', 'landuseavg', 'numfloorsavg']], data.avg_health_round, test_size=0.20, random_state=0)
print(X_train)

      zipcode  spc_latin  tree_diameter   latBin   lonBin  wires  \
2708    10023        470              6  40.7715 -73.9850      0   
2077    10016        113             10  40.7445 -73.9800      0   
1564    10010        683              3  40.7355 -73.9765      0   
411     10001        470              8  40.7500 -73.9965      0   
2437    10021        683             10  40.7675 -73.9535      0   
...       ...        ...            ...      ...      ...    ...   
763     10002         69              5  40.7170 -73.9885      0   
835     10003         24              6  40.7340 -73.9890      0   
1653    10011        113              5  40.7335 -73.9955      0   
2607    10023         59             14  40.7735 -73.9815      0   
2732    10025        299              5  40.8045 -73.9630      0   

      sidew_crack_raise  landuseavg  numfloorsavg  
2708                  0         7.0          19.0  
2077                  0         4.0          31.0  
1564                  0    

In [147]:
# Check how unbalanced dataset is
# data.avg_health.value_counts()
# data.avg_health_round.value_counts()
print(Counter(y_train))

Counter({2.0: 1447, 3.0: 918, 1.0: 102, 0.0: 6})


In [148]:
# Try random oversampling
ros = RandomOverSampler()
X_train, y_train = ros.fit_resample(X_train, y_train)
print(Counter(y_train))

Counter({2.0: 1447, 3.0: 1447, 1.0: 1447, 0.0: 1447})


In [36]:
# Try random undersampling (this performed worse)
# rus = RandomUnderSampler()
# X_train, y_train = rus.fit_resample(X_train, y_train)
# print(Counter(y_train))

Counter({0.0: 2336, 1.0: 2336, 2.0: 2336, 3.0: 2336})


# Classification


In [17]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)


/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [127]:
predictions = log_reg.predict(X_test)
report = sklearn.metrics.classification_report(y_test, predictions)
print(report)

/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.60      0.01       603
         1.0       0.02      0.11      0.04      5173
         2.0       0.56      0.36      0.43    103354
         3.0       0.00      0.00      0.00     92642

    accuracy                           0.19    201772
   macro avg       0.15      0.27      0.12    201772
weighted avg       0.29      0.19      0.22    201772



/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [128]:
# Feature influence
print('Feature influence')
print(log_reg.coef_)
# print(np.std(X_train, 0)*log_ref.coef_)


Feature influence
[[ 8.93830071e-06 -1.19162890e-06  1.49081186e-08  3.18664141e-08
  -5.73179965e-08  7.92462037e-10  7.17089123e-10]
 [-6.62261236e-06  8.94516213e-07 -2.36067606e-08 -2.48173762e-08
   4.52883483e-08 -8.40244077e-10 -1.26547880e-09]
 [-6.92264075e-06  9.25343723e-07 -1.69611846e-08 -2.53794632e-08
   4.57695693e-08 -6.67470916e-10 -1.01184263e-09]
 [ 4.60695241e-06 -6.28231035e-07  2.56598265e-08  1.83304253e-08
  -3.37399210e-08  7.15252955e-10  1.56023230e-09]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest=RandomForestClassifier(n_estimators=1000)
random_forest.fit(X_train,y_train)

In [155]:
# Eval
predictions = random_forest.predict(X_test)
report = sklearn.metrics.classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

         1.0       0.94      0.85      0.89        20
         2.0       0.93      0.94      0.94       363
         3.0       0.92      0.90      0.91       236

    accuracy                           0.93       619
   macro avg       0.93      0.90      0.91       619
weighted avg       0.93      0.93      0.93       619



In [93]:
# Grid search cross validation
# from sklearn.model_selection import GridSearchCV

# param_grid = {'n_estimators': [100,500,1000], 'criterion': ['entropy', 'gini']}

# grid_clf = GridSearchCV(random_forest, param_grid, scoring=['f1_weighted', 'f1_macro','accuracy'], refit='accuracy', cv=3)
# grid_clf.fit(X_train, y_train)


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(criterion='entropy',
                                              n_estimators=50),
             param_grid={'criterion': ['entropy', 'gini'],
                         'n_estimators': [100, 500, 1000]},
             refit='accuracy', scoring=['f1_weighted', 'f1_macro', 'accuracy'])

In [133]:
# Hyperparam grid search results and eval
# grid_clf.cv_results_
# grid_clf.best_estimator_
# grid_clf.best_score_

# import pickle
# filename = 'grid_clf.best_estimator_.sav'
# pickle.dump(grid_clf.best_estimator_, open(filename, 'wb'))

# predictions = grid_clf.best_estimator_.predict(X_train)
# predictions = random_forest.predict(X_train)
# # report = sklearn.metrics.classification_report(y_test, predictions)
# # print(report)
# grid_clf.best_estimator_.score(X_test, y_test)
# sklearn.metrics.accuracy_score(y_test, predictions)


In [156]:
# Feature influence
# From sklearn: The higher, the more important the feature. 
# The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. 
# It is also known as the Gini importance.

random_forest.feature_importances_

#From first features: array([0.05849014, 0.09828617, 0.15330729, 0.33727531, 0.35264109])
# This shows that lat and lon bin are the most important
# Zip code and species are least important


#With PLUTO data: array([0.14133116, 0.12636846, 0.08450248, 0.20923166, 0.16384807, 0.0844982 , 0.19021996])
#In order of importance: latbin, numfloors, lonbin, zipcode, spc, landuse, diameter

#With PLUTO data and wires and sidewalk: 
# array([0.14362981, 0.12422975, 0.07294823, 0.22162114, 0.14892171, 0.00399293, 0.01116889, 0.08029825, 0.19318929])
# In order of importance: lat, numfloors, lon, zipcode, spc, landuse, diameter, sidewalk, wires
# With 1000 trees: array([0.14992811, 0.12930077, 0.07363184, 0.20934803, 0.15155635, 0.00338376, 0.01061805, 0.08095947, 0.19127362])
# lat, numfloors, lon, zipcode, spc, landuse, diam, sidewalk, wires

# Same as above but without lat and lon bin:
#array([0.26028929, 0.16251933, 0.1134749 , 0.00598594, 0.01296771, 0.1436791 , 0.30108372])
# numfloors, zipcode, landuse, spc, diam, sidewalk, wires

# Same as above but without zipcode either:
# array([0.1993551 , 0.14890345, 0.00593354, 0.01365038, 0.20375706, 0.42840046])
# numfloors, landuse, spc, diameter, sidewalks, wires

# array([0.4826821 , 0.45951688, 0.02285163, 0.0349494 ]) spc, diam, sidewalk, wires

array([0.15157782, 0.12423122, 0.07510414, 0.20895941, 0.14897492,
       0.00340121, 0.01158761, 0.08171396, 0.19444972])

In [114]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
gradient_boosting = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0, max_depth=8, random_state=0)
gradient_boosting.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=8, n_estimators=500,
                           random_state=0)

In [ ]:
# Grid search cross validation
# from sklearn.model_selection import GridSearchCV

# param_grid = {'n_estimators': [100,500,1000], 'learning_rate': [0.001, 0.01, 0.1, 0.2]}

# grid_clf = GridSearchCV(gradient_boosting, param_grid, scoring=['f1_weighted', 'f1_macro','accuracy'], refit='accuracy', cv=3)
# grid_clf.fit(X_train, y_train)

In [115]:
# Eval
predictions = gradient_boosting.predict(X_test)
report = sklearn.metrics.classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

         1.0       0.94      0.85      0.89        20
         2.0       0.94      0.96      0.95       363
         3.0       0.94      0.91      0.92       236

    accuracy                           0.94       619
   macro avg       0.94      0.91      0.92       619
weighted avg       0.94      0.94      0.94       619



In [110]:
# SVM
from sklearn.svm import SVC
svm = SVC(gamma='auto', C=3, random_state=0)
svm.fit(X_train, y_train)

SVC(C=3, cache_size=500, gamma='auto', random_state=0)

In [111]:
# Eval
predictions = svm.predict(X_test)
report = sklearn.metrics.classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       0.88      0.75      0.81        20
         2.0       0.88      0.92      0.90       363
         3.0       0.88      0.82      0.85       236

    accuracy                           0.88       619
   macro avg       0.66      0.62      0.64       619
weighted avg       0.88      0.88      0.88       619



/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

# Regression
### (This did not work well)

In [127]:
X_train, X_test, y_train, y_test = train_test_split(data[['zipcode', 'spc_latin', 'tree_diameter', 'latBin', 'lonBin']], data.avg_health, test_size=0.20, random_state=0)

In [128]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression(fit_intercept=True)
lin_reg.fit(X_train, y_train)


LinearRegression()

In [129]:
predictions = lin_reg.predict(X_test)
print("Score")
print(lin_reg.score(X_test, y_test))

print('Feature influence')
print(lin_reg.coef_)
print(np.std(X_train, 0)*lin_reg.coef_)


Score
0.011350976452213057
Feature influence
[ 2.14129907e-05 -5.64659458e-07  4.75334829e-03  1.82898365e-01
 -3.07816016e-01]
zipcode          0.010920
spc_latin       -0.035764
tree_diameter    0.045405
latBin           0.016108
lonBin          -0.039221
dtype: float64


In [93]:
from sklearn.ensemble import RandomForestRegressor
random_forest_reg=RandomForestRegressor(n_estimators=100)
random_forest_reg.fit(X_train,y_train)

RandomForestRegressor()

In [109]:
print("Score")
random_forest_reg.score(X_test, y_test)


Score


0.4181915364123192

# Validation Experiments on Best Models

In [20]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
ros = RandomOverSampler()

rf_accuracies = []
gb_accuracies = []
rf_f1_macro = []
gb_f1_macro = []
rf_f1_weighted = []
gb_f1_weighted = []

# Repeat a different train/test split 5 times to validate best model performance
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(data[['zipcode', 'spc_latin', 'tree_diameter', 'latBin', 'lonBin', 'wires', 'sidew_crack_raise', 'landuseavg', 'numfloorsavg']], data.avg_health_round, test_size=0.20, random_state=None)
    X_train, y_train = ros.fit_resample(X_train, y_train)

    random_forest=RandomForestClassifier(n_estimators=1000)
    random_forest.fit(X_train,y_train)

    gradient_boosting = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, max_depth=10)
    gradient_boosting.fit(X_train, y_train)

    predictions_rf = random_forest.predict(X_test)
    predictions_gb = gradient_boosting.predict(X_test)


    report = sklearn.metrics.classification_report(y_test, predictions_rf)
    print("Random Forest ", i)
    print(report)
    rf_accuracies.append(sklearn.metrics.accuracy_score(y_test, predictions_rf))
    rf_f1_macro.append(sklearn.metrics.f1_score(y_test, predictions_rf, average='macro'))
    rf_f1_weighted.append(sklearn.metrics.f1_score(y_test, predictions_rf, average='weighted'))

    report = sklearn.metrics.classification_report(y_test, predictions_gb)
    print("Gradient Boosting ", i)
    print(report)
    gb_accuracies.append(sklearn.metrics.accuracy_score(y_test, predictions_gb))
    gb_f1_macro.append(sklearn.metrics.f1_score(y_test, predictions_gb, average='macro'))
    gb_f1_weighted.append(sklearn.metrics.f1_score(y_test, predictions_gb, average='weighted'))

print("Accuracy Scores")
print(np.mean(rf_accuracies))
print(np.mean(gb_accuracies))
print("F1 macro scores")
print(np.mean(rf_f1_macro))
print(np.mean(gb_f1_macro))
print("F1 Weighted Scores")
print(np.mean(rf_f1_weighted))
print(np.mean(gb_f1_weighted))

print('Standard deviations')
print(np.std(rf_accuracies))
print(np.std(gb_accuracies))
print(np.std(rf_f1_macro))
print(np.std(gb_f1_macro))
print(np.std(rf_f1_weighted))
print(np.std(gb_f1_weighted))







Random Forest  0
              precision    recall  f1-score   support

         1.0       0.96      0.92      0.94        26
         2.0       0.92      0.91      0.92       352
         3.0       0.88      0.90      0.89       241

    accuracy                           0.91       619
   macro avg       0.92      0.91      0.92       619
weighted avg       0.91      0.91      0.91       619

Gradient Boosting  0
              precision    recall  f1-score   support

         1.0       1.00      0.88      0.94        26
         2.0       0.92      0.93      0.93       352
         3.0       0.90      0.89      0.90       241

    accuracy                           0.92       619
   macro avg       0.94      0.90      0.92       619
weighted avg       0.92      0.92      0.92       619

Random Forest  1
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1
         1.0       0.92      0.86      0.89        28
         2.0       0

/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

Random Forest  3
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.85      0.96      0.90        24
         2.0       0.93      0.93      0.93       359
         3.0       0.91      0.89      0.90       235

    accuracy                           0.92       619
   macro avg       0.67      0.70      0.68       619
weighted avg       0.91      0.92      0.92       619

Gradient Boosting  3
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.81      0.88      0.84        24
         2.0       0.92      0.92      0.92       359
         3.0       0.90      0.89      0.90       235

    accuracy                           0.91       619
   macro avg       0.66      0.67      0.66       619
weighted avg       0.91      0.91      0.91       619

Random Forest  4
              precision    recall  f1-score   support

         1.0       0